In [1]:
import os
import sys

In [2]:
import json
import re

In [3]:
_PKG_PATH: str = "/Users/adebayobraimah/Desktop/projects/CSE505"
sys.path.append(_PKG_PATH)

In [4]:
# from src.clapi.clapi import _write_list_to_file, translate_range
from src.clapi.clapi import process_course_data_clingo

In [5]:
json_file = (
    "/Users/adebayobraimah/Desktop/projects/CSE505/src/resources/cse_courses.json"
)

In [6]:
# def generate_clingo_predicates_rules(json_file):
#     with open(json_file, "r") as file:
#         data = json.load(file)

#     predicates = []
#     rules = []

#     output_file = json_file.replace(".json", ".lp")

#     for course_id, course_info in data.items():
#         # Generate course predicates
#         offered_spring1 = int(course_info["spring1"])
#         offered_fall1 = int(course_info["fall1"])
#         offered_spring2 = int(course_info["spring2"])
#         offered_fall2 = int(course_info["fall2"])

#         credits = (
#             int(course_info.get("Credits"))
#             if isinstance(course_info.get("Credits"), (float, int))
#             else _translate_range(course_info.get("Credits"))
#         )

#         career = course_info.get("Career")

#         predicates.append(
#             f'course({course_id.lower()}, {credits}, "{career}", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}).'
#         )

#         # Generate antirequisite rules
#         if course_info["Antirequisites"] != "NONE":
#             rules.append(f"% Antirequisites for {course_id.upper()}")
#             for antireq_group in course_info["Antirequisites"]:
#                 for antireq in antireq_group:
#                     rules.append(
#                         f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), \n   course({antireq.lower()}, _, \"{career}\", _, _, _, _). \n"
#                     )

#         # Generate prerequisite rules
#         if course_info["Prerequisites"] != "NONE":
#             rules.append(f"% Prerequisites for {course_id.upper()}")
#             for prereq_group in course_info["Prerequisites"]:
#                 group_conditions = ", ".join(
#                     [
#                         f'\n   not course({prereq.lower()}, _, "{career}", _, _, _, _)'
#                         for prereq in prereq_group
#                     ]
#                 )
#                 rules.append(
#                     f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), {group_conditions}. \n" 
#                 )

#         # Generate corequisite rules
#         if course_info["Corequisites"] != "NONE":
#             rules.append(f"% Corequisites for {course_id.upper()}")
#             for coreq_group in course_info["Corequisites"]:
#                 for coreq in coreq_group:
#                     rules.append(
#                         f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), \n   not course({coreq.lower()}, _, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}). \n"
#                     )

#     # Write predicates and rules to file
#     output = (
#         ["% Course predicates"] + predicates + ["\n% Course Rules \n"] + rules
#     )
#     _write_list_to_file(output, output_file)

#     # Output results
#     return {"predicates": predicates, "rules": rules}


# # Usage
# results = generate_clingo_predicates_rules(json_file)
# # print("Predicates:")
# # for predicate in results["predicates"]:
# #     print(predicate)
# # print("\nRules:")
# # for rule in results["rules"]:
# #     print(rule)

In [7]:
# results

In [8]:
process_course_data_clingo(json_file,repeatable_courses=None)

'/Users/adebayobraimah/Desktop/projects/CSE505/src/resources/cse_courses.lp'

In [9]:
# def create_clingo_atoms_from_json(file_path):
#     # Open and read the JSON file
#     with open(file_path, "r") as file:
#         data = json.load(file)

#     # List to store the clingo atoms
#     clingo_atoms = []

#     # Process each course
#     for course_id, course_info in data.items():
#         description = course_info.get("Description", "")

#         # Check if the course is repeatable
#         if "repeat" in description.lower():
#             # Initialize min_credits and max_credits
#             min_credits = "_"
#             max_credits = "_"

#             # Example logic to extract credits info; this will need customization based on actual data
#             # For simplicity, let's assume we find sentences like "May be repeated for up to X credits."
#             words = description.split()
#             if "credits" in words:
#                 credit_index = words.index("credits")
#                 for i in range(credit_index, credit_index - 4, -1):
#                     if words[i - 1].isdigit():
#                         max_credits = words[i - 1]
#                         break

#             # Create the clingo atom string
#             atom = f"repeatable({course_id.lower()}, {min_credits}, {max_credits})"
#             clingo_atoms.append(atom)

#     return clingo_atoms


# # Example usage
# file_path = json_file
# clingo_atoms = create_clingo_atoms_from_json(file_path)
# # for atom in clingo_atoms:
# #     print(atom)

In [10]:
# clingo_atoms

In [11]:
# def extract_credits(description):
#     # Search for one or more digits followed by the word 'credits'
#     match = re.search(r"(\d+) credits", description)
#     if match:
#         # Return the number found
#         return int(match.group(1))
#     else:
#         return None

In [12]:
# def _check_repeatable_input(course_info):
#     try:
#         int(course_info)
#         return True
#     except ValueError:
#         if course_info == "_":
#             return True
#         else:
#             return False

In [13]:
# def generate_atoms(json_file, other_courses=None):
#     with open(json_file, "r") as file:
#         data = json.load(file)

#     atoms = []

#     # other_courses = [('cse593',_,_)]
#     if other_courses is not None:
#         for course_info in other_courses:
#             if (not course_info[0].upper() in data.keys()):
#                 raise ValueError(f"Course {course_info[0]} not found in the JSON file.")
#             if (len(course_info) != 3) or (not _check_repeatable_input(course_info[1])) or (not _check_repeatable_input(course_info[2])):
#                 raise ValueError("Each course tuple should have exactly 3 elements.")
#             atom = f"repeatable({course_info[0]}, {course_info[1]}, {course_info[2]})."
#             atoms.append(atom)

#     for course_id, course_info in data.items():
#         description = course_info["Description"].lower()
#         if ("repeat" in description) and (not course_id in atoms):
#             times = "_"
#             max_credits = "_"
#             if "more than twice" in description:
#                 times = 2
#             elif "repeated once" in description:
#                 times = 2
#             elif "repeated twice" in description:
#                 times = 3
#             elif ("credits" in description.lower()) and (extract_credits(description) is not None):
#                 max_credits = extract_credits(description)
#             else:
#                 # NOTE: Uncommenting the line below will set the default value of times to 2.
#                 #   This behavior may not be desired in all cases.
#                 # times: int = 2
#                 continue
#             atom = f"repeatable({course_id.lower()}, {times}, {max_credits})."
#             atoms.append(atom)
#     atoms = sorted(atoms)
#     return atoms


# # Use the actual data structure from the file you uploaded
# generate_atoms(json_file, [("cse593", '_', '_'),('cse390', 2, 4)])

In [14]:
# _check_repeatable_input("_")